In [168]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [78]:
traindata = pd.read_csv(r"E:\datasets\facebook lr+gbdt\gbdt-lr-master\data\train.csv")

In [79]:
trainset, testset = train_test_split(traindata, test_size = 0.2)

In [166]:
X_train_num = trainset.iloc[:,2:15]
X_train_cat = trainset.iloc[:,16:]
y_train = trainset['Label']

X_test_num = testset.iloc[:,2:15]
X_test_cat = testset.iloc[:,16:]
y_test = testset['Label']

lgb_dataset = lgb.Dataset(X_train_num,y_train)

In [225]:
param_dist ={
    "learning_rate" : [0.01,0.05,0.1],
    "num_leaves": [32,64],
    "reg_alpha":[1,5,50,100],
    "reg_lambda":[0.1,0.5,1,5]
}

lgbclf = lgb.LGBMClassifier(silent=False)
grid_search = GridSearchCV(lgbclf, n_jobs=-1, param_grid=param_dist, cv = 5, scoring="roc_auc", verbose=5)
grid_search.fit(X_train_num,y_train)
grid_search.best_estimator_

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 469 out of 480 | elapsed:   21.6s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   21.7s finished


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=32, objective=None,
        random_state=None, reg_alpha=5, reg_lambda=5, silent=False,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [264]:
params = {'objective':'binary','learning_rate':0.1,'num_leaves':32,
          'reg_alpha':5, 'reg_lambda':5}
lgb_model = lgb.LGBMClassifier(**params)
lgb_model.fit(X_train_num,y_train,eval_set=(X_train_num,y_train),verbose=10)

[10]	training's binary_logloss: 0.451723
[20]	training's binary_logloss: 0.427524
[30]	training's binary_logloss: 0.406907
[40]	training's binary_logloss: 0.388239
[50]	training's binary_logloss: 0.371055
[60]	training's binary_logloss: 0.359256
[70]	training's binary_logloss: 0.34893
[80]	training's binary_logloss: 0.343071
[90]	training's binary_logloss: 0.343067
[100]	training's binary_logloss: 0.343065


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=32, objective='binary',
        random_state=None, reg_alpha=5, reg_lambda=5, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [269]:
y_pred_leaf = lgb_model.predict(X_train_num,pred_leaf=True)

In [265]:
lgbclf_ll = log_loss(y_test,lgbclf_pred)
lgbclf_ll

8.310918057445075

In [270]:
enc = OneHotEncoder()
tree_leaf_enc = enc.fit_transform(y_pred_leaf).toarray()

In [276]:
tree_leaf_enc.shape

(1279, 1566)

In [271]:
lr = LogisticRegression()
lr.fit(tree_leaf_enc,y_train)
gbdt_lr_pred = lr.predict_proba(tree_leaf_enc)

In [291]:
np.round(gbdt_lr_pred,3)[:,0]

array([0.796, 0.98 , 0.947, ..., 0.999, 0.999, 0.928])

In [285]:
final_result = trainset.iloc[:,0:2]

In [292]:
final_result['ranking'] = np.round(gbdt_lr_pred,3)[:,0]

In [293]:
final_result

,Id,Label,ranking
522,10000320,0,0.796
1275,10000804,0,0.980
889,10000747,0,0.947
489,10001806,0,1.000
1393,10000889,0,0.924
524,10001227,0,0.995
1596,10001653,0,1.000
494,10000913,1,0.209
960,10001734,0,1.000
1469,10001634,0,0.934


In [297]:
final_result.sort_values(by='ranking',ascending=False)

,Id,Label,ranking
1575,10000174,0,1.000
1430,10001104,0,1.000
477,10000752,0,1.000
541,10000064,0,1.000
895,10001957,0,1.000
652,10001759,0,1.000
515,10001522,0,1.000
882,10001781,0,1.000
961,10000125,0,1.000
153,10001334,0,1.000
